In [1]:
import backtrader as bt
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [70]:
# class VWAP(bt.Indicator):
#     lines = ('vwap',)
#     plotinfo = dict(subplot=False)
    
#     def next(self):
#         # Calculate cumulative sum of price*volume and volume up to the current bar
#         total_pv = sum(self.data.close[i] * self.data.volume[i] for i in range(len(self.data)))
#         total_vol = sum(self.data.volume[i] for i in range(len(self.data)))
#         self.lines.vwap[0] = total_pv / total_vol if total_vol != 0 else self.data.close[0]


class VWAPRollingIndicator(bt.Indicator):
    """
    Volume Weighted Average Price (VWAP) indicator, rolling calculation.
    """

    lines = ("vwap_rolling",)
    params = {"period": 14}
    plotinfo = {"subplot": False}
    plotlines = {"vwap_rolling": {"color": "green"}}

    def __init__(self) -> None:
        self.hlc = (self.data.high + self.data.low + self.data.close) / 3.0
        self.hlc_volume_sum = bt.ind.SumN(self.hlc * self.data.volume, period=self.p.period)
        self.volume_sum = bt.ind.SumN(self.data.volume, period=self.p.period)

        self.lines.vwap_rolling = bt.DivByZero(self.hlc_volume_sum, self.volume_sum, None) 


        
class VWAPStrategy(bt.Strategy):
    params = (
        ("stop_loss", 0.02),  # 2% stop loss
        ("take_profit", 0.04),  # 4% take profit
        ("risk", 0.01),  # 1% risk
        ("pivot_window", 5),  # 5 bars for pivot high/low
        ("rrr", 2),  # Risk-Reward Ratio
    )
    def __init__(self):
        # initialize the VWAP indicator on data feed
        self.vwap = VWAPRollingIndicator(self.data)
        self.crossover = bt.indicators.CrossOver(self.data.close, self.vwap)
        self.order = None
        self.prev_pivot_high = self.data.low[0]
        self.prev_pivot_low = self.data.high[0]
        
    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
            elif order.issell():
                self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
        self.order = None
    
    def log(self, txt):
        dt = self.data.datetime.date(0)
        print(f'{dt.isoformat()} {txt}')
    
    def _calculate_size(self, price, sl_price):
        # Calculate the size of the position based on the risk percentage and stop loss
        size = (self.broker.getvalue() * self.params.risk) / abs(price - sl_price)
        return size
    
    def _update_pivots(self):
        if np.max(self.data.high.get(size=self.params.pivot_window * 2 + 1)) == self.data.high[self.params.pivot_window]:
            self.prev_pivot_high = self.data.high[self.params.pivot_window]   
        if np.min(self.data.low.get(size=self.params.pivot_window * 2 + 1)) == self.data.low[self.params.pivot_window]:
            self.prev_pivot_low = self.data.low[self.params.pivot_window]      

    def next(self):
        
        # if not in a market position, check for crossover from below
        if not self.position:
            if self.crossover > 0 and self.data.close[1] > self.data.open[1]:
                price = self.data.close[0]
                sl_price = self.data.low[1]
                tp_price = price - (sl_price - price) * self.params.rrr
                if tp_price < price:
                    self.log(f'Take profit price {tp_price:.2f} is less than entry price {price:.2f}, not placing order.')
                    return
                if sl_price > price:
                    self.log(f'Stop loss price {sl_price:.2f} is greater than entry price {price:.2f}, not placing order.')
                    return

                size = self._calculate_size(price, sl_price)
                self.order = self.buy_bracket(
                    exectype=bt.Order.Market, 
                    size=size,
                    price=price,
                    stopprice=sl_price,
                    limitprice=tp_price,
                    stopargs=dict(exectype=bt.Order.Stop),
                    limitargs=dict(exectype=bt.Order.Limit)
                )
                self.log(f"Bracket order placed: Entry {price:.2f}, SL {sl_price:.2f}, TP {tp_price:.2f}")
 
            # if in a position, check for crossover from above for exit
            elif self.crossover < 0 and self.data.close[1] < self.data.open[1]:
                price = self.data.close[0]
                sl_price = self.data.high[1]
                tp_price = price - (sl_price - price) * self.params.rrr
                if tp_price > price:
                    self.log(f'Take profit price {tp_price:.2f} is greater than entry price {price:.2f}, not placing order.')
                    return
                if sl_price < price:
                    self.log(f'Stop loss price {sl_price:.2f} is less than entry price {price:.2f}, not placing order.')
                    return
                size = self._calculate_size(price, sl_price)
                self.order = self.sell_bracket(
                    exectype=bt.Order.Market, 
                    size=size,
                    price=price,
                    stopprice=sl_price,
                    limitprice=tp_price,
                    stopargs=dict(exectype=bt.Order.Stop),
                    limitargs=dict(exectype=bt.Order.Limit)
                )
                self.log(f"Bracket order placed: Entry {price:.2f}, SL {sl_price:.2f}, TP {tp_price:.2f}")
        else:
            pass
            


# class VWAPBracketStrategy(bt.Strategy):
#     params = (
#         ("stop_loss", 0.02),    # 2% stop loss
#         ("take_profit", 0.04),  # 4% take profit
#         ("risk", 0.01),         # 1% risk
#         ("pivot_window", 5),    # 5 bars for pivot high/low (unused here)
#         ("rrr", 2),             # Risk-Reward Ratio (unused here)
#     )
    
#     def __init__(self):
#         self.vwap = VWAPRollingIndicator(self.data)
#         self.crossover = bt.indicators.CrossOver(self.data.close, self.vwap)
#         self.order = None
        
#     def notify_order(self, order):
#         if order.status in [order.Completed]:
#             if order.isbuy():
#                 self.log(f"BUY EXECUTED, {order.executed.price:.2f}")
#             elif order.issell():
#                 self.log(f"SELL EXECUTED, {order.executed.price:.2f}")
#         elif order.status in [order.Canceled, order.Margin, order.Rejected]:
#             self.log("Order Canceled/Margin/Rejected")
#         self.order = None
        
#     def log(self, txt):
#         dt = self.data.datetime.date(0)
#         print(f"{dt.isoformat()} {txt}")
    
#     def _calculate_size(self, price, sl_price):
#         # Calculate the position size based on the risk per trade.
#         return (self.broker.getvalue() * self.p.risk) / abs(price - sl_price)
    
#     def next(self):
#         # When not in a position, check for entry signal
#         if not self.position:
#             if self.crossover > 0:
#                 price = self.data.close[0]
#                 # Define stop loss as a percentage below the entry price
#                 sl_price = price * (1 - self.p.stop_loss)
#                 # Define take profit as a percentage above the entry price
#                 tp_price = price * (1 + self.p.take_profit)
#                 size = self._calculate_size(price, sl_price)
                
#                 # Place a bracket order with entry, stop loss, and take profit orders
#                 self.order = self.buy_bracket(
#                     size=size,
#                     price=price,
#                     stopprice=sl_price,
#                     limitprice=tp_price,
#                     exectype=bt.Order.Market,
#                     stopargs=dict(exectype=bt.Order.Stop),
#                     limitargs=dict(exectype=bt.Order.Limit)
#                 )
#                 self.log(f"Bracket order placed: Entry {price:.2f}, SL {sl_price:.2f}, TP {tp_price:.2f}")
#         else:
#             # If a crossover reversal is detected, exit the current position.
#             if self.crossover < 0:
#                 self.close()
#                 self.log("Exiting position due to crossover reversal")

In [71]:
# Download SPX500 data from Yahoo Finance
yf_params = {'start': '2018-01-01', 'end': '2024-01-01', 'interval': '1d', 'multi_level_index': False}
data = yf.download(tickers="^GSPC", **yf_params)
# data = yf.download(tickers="DJIA", **yf_params)
data.reset_index(inplace=True)
data['Date'] = pd.to_datetime(data['Date'])
# data.set_index('Date', inplace=True)

# data

[*********************100%***********************]  1 of 1 completed


In [75]:
cerebro = bt.Cerebro()
cerebro.addstrategy(VWAPStrategy, stop_loss=0.02, take_profit=0.04, risk=0.01, pivot_window=5, rrr=2)
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.001)
data_feed = bt.feeds.pandafeed.PandasData(
    dataname=data,
    datetime=0,
    openinterest=-1,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
)
cerebro.adddata(data_feed)

In [76]:
cerebro.addsizer(bt.sizers.FixedSize, stake=10)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.01)
cerebro.addanalyzer(bt.analyzers.DrawDown)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)
cerebro.addanalyzer(bt.analyzers.Returns)

results = cerebro.run()

2018-02-16 Bracket order placed: Entry 2727.14, SL 2706.76, TP 2767.90
2018-03-02 Bracket order placed: Entry 2658.89, SL 2728.09, TP 2520.49
2018-03-05 SELL EXECUTED, 2720.94
2018-03-06 BUY EXECUTED, 2728.12
2018-03-07 Bracket order placed: Entry 2710.18, SL 2740.45, TP 2649.64
2018-03-08 SELL EXECUTED, 2738.97
2018-03-09 BUY EXECUTED, 2786.57
2018-04-10 Take profit price 2636.73 is less than entry price 2638.41, not placing order.
2018-04-25 Bracket order placed: Entry 2634.92, SL 2676.48, TP 2551.80
2018-04-26 SELL EXECUTED, 2666.94
2018-04-27 BUY EXECUTED, 2676.48
2018-04-27 Bracket order placed: Entry 2675.47, SL 2648.04, TP 2730.33
2018-04-30 BUY EXECUTED, 2648.05
2018-05-01 SELL EXECUTED, 2648.04
2018-05-29 Bracket order placed: Entry 2705.11, SL 2729.34, TP 2656.65
2018-06-01 Bracket order placed: Entry 2718.70, SL 2749.16, TP 2657.78
2018-06-04 SELL EXECUTED, 2746.87
2018-06-05 BUY EXECUTED, 2749.16
2018-06-20 Bracket order placed: Entry 2769.73, SL 2744.39, TP 2820.41
2018-08

In [77]:
strat = results[0]
sharpe_ratio = strat.analyzers.sharperatio.get_analysis()
drawdown = strat.analyzers.drawdown.get_analysis()
trade_analysis = strat.analyzers.tradeanalyzer.get_analysis()
returns = strat.analyzers.returns.get_analysis()
print(f'Sharpe Ratio: {sharpe_ratio}')
print(f'Max Drawdown: {drawdown["max"]["drawdown"]}')
print(f'Number of Trades: {trade_analysis.total.closed}')
print(f'Winning Trades: {trade_analysis.won.total}')
print(f'Losing Trades: {trade_analysis.lost.total}')
print(f'Average Trade Return: {returns["rnorm"]}')

Sharpe Ratio: OrderedDict({'sharperatio': -2.5211144440137443})
Max Drawdown: 14.29394026763638
Number of Trades: 38
Winning Trades: 2
Losing Trades: 36
Average Trade Return: -0.01914422669972353
